In [1]:
import os  # Import the os library
print("Current Working Directory:", os.getcwd())

Current Working Directory: c:\Python


In [2]:
os.chdir("C:/Python")

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lets_plot import *
from scipy import stats
import statsmodels.api as sm 

# before using lets_plot you need to call the setup
LetsPlot.setup_html(no_js=True)

In [4]:
! pip install lets_plot

In [5]:
fes_data = pd.read_stata("datafes.dta")
fes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189671 entries, 0 to 189670
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   year      189656 non-null  float64 
 1   age       189656 non-null  float64 
 2   coh       189671 non-null  category
 3   famtype   189656 non-null  category
 4   totexp    189656 non-null  float32 
 5   totexpeq  189656 non-null  float32 
 6   totnd     189656 non-null  float32 
 7   totndeq   189656 non-null  float32 
 8   income    187842 non-null  float32 
dtypes: category(2), float32(5), float64(2)
memory usage: 6.9 MB


In [6]:
fes_data.head()

,year,age,coh,famtype,totexp,totexpeq,totnd,totndeq,income
0,74.0,41.0,born 1930-39,10 Extended family plus children,565.546204,137.938095,370.877502,90.457932,620.269775
1,74.0,61.0,born 1910-19,5 Couple,173.752411,102.207298,121.898033,71.704727,316.160767
2,74.0,82.0,born 1900-,1 Single,83.631500,83.631500,41.325153,41.325153,72.689499
3,74.0,41.0,born 1930-39,6 Couple plus dependent children,300.285736,111.216934,194.519867,72.044395,342.416138
4,74.0,40.0,born 1930-39,2 Lone plus dependent children,238.775070,119.387535,168.237427,84.118713,259.332764


In [7]:
fes_data['famtype'].value_counts()

famtype
5  Couple                                   50651
6  Couple plus dependent children           46666
1  Single                                   46040
7  Couple plus non dependent children       10653
2  Lone plus dependent children              8262
9  Extended family                           8139
8  Couple plus both                          7313
11 Other multiple tax unit                   3499
10 Extended family plus children             3368
3  Lone plus non dependent children          3063
4  Lone plus both                            1307
12 Other multiple tax unit plus children      695
Name: count, dtype: int64

In [8]:
fes_data_couple = fes_data.query('famtype in ["5  Couple                               ","7  Couple plus non dependent children   "]').copy()

In [9]:
fes_data_couple['coh'].value_counts()

coh
born 1920-29    15095
born 1930-39    11107
born 1910-19    10687
born 1940-49     8269
born 1950-59     5877
born 1900-09     4623
born 1960-69     3872
born 1970+       1070
born 1900-        704
Name: count, dtype: int64

In [10]:
fes_data_sel = fes_data_couple.query('coh in ["born 1930-39","born 1940-49","born 1950-59"]').copy()
fes_data_sel['coh'].value_counts()

coh
born 1930-39    11107
born 1940-49     8269
born 1950-59     5877
born 1970+          0
born 1960-69        0
born 1920-29        0
born 1910-19        0
born 1900-09        0
born 1900-          0
Name: count, dtype: int64

In [11]:
fes_data_sel['income'].describe()

count    24943.000000
mean       623.546997
std        453.475769
min          0.000000
25%        380.823547
50%        549.334656
75%        768.025391
max      18904.652344
Name: income, dtype: float64

In [12]:
p1 = ggplot(fes_data_sel, aes(x = 'income')) + geom_histogram(binwidth = 500)
p1

In [13]:
p2 = (ggplot(fes_data_sel, aes(x = 'income')) + 
      geom_histogram(binwidth = 100) + 
      xlim(0,5000) +
      labs(title = "Histogram, income, max 5000"))
p2

In [14]:
fes_data_sel = fes_data_sel.query('income > 0').copy()
fes_data_sel['lincome'] = np.log(fes_data_sel['income'])
fes_data_sel['lincome'].describe()

count    24937.000000
mean         6.256570
std          0.625843
min         -3.159846
25%          5.942802
50%          6.308816
75%          6.643944
max          9.847163
Name: lincome, dtype: float64

In [15]:
p3 = (ggplot(fes_data_sel, aes(x = 'lincome')) + 
      geom_histogram(binwidth = 0.1) + 
      labs(title = "Histogram, ln(income)"))
p3

In [16]:
mu = np.mean(fes_data_sel['lincome'])  # mean estimate
sig2 = np.std((fes_data_sel['lincome']))**2 # variance estimate
print("sample mean = ", round(mu,2), ", Sample variance =", round(sig2,2))

sample mean =  6.26 , Sample variance = 0.39


In [32]:
lx_range = np.arange(-2,10,0.01)
lx_pdf = (1/(np.sqrt(2*np.pi*sig2)))*np.exp(-(lx_range-mu)**2/(2*sig2))
lx_pdf_plot = {'lincome': lx_range,
                'lx_pdf': lx_pdf}
lx_pdf_plot = pd.DataFrame(lx_pdf_plot)

In [34]:
p4 = (ggplot(lx_pdf_plot, aes(x = 'lincome', y = 'lx_pdf')) + 
      geom_line() + 
      labs(title = "ln(income), ML estimate N"))
p4

In [35]:
x_range = np.arange(10,5000,10)
x_pdf = (1/(x_range*np.sqrt(2*np.pi*sig2)))*np.exp(-(np.log(x_range)-mu)**2/(2*sig2))
x_pdf_plot = {'income': x_range,
        'x_pdf': x_pdf}
x_pdf_plot = pd.DataFrame(x_pdf_plot)

In [36]:
p5 = (ggplot(x_pdf_plot, aes(x = 'income', y = 'x_pdf')) + 
      geom_line() + 
      labs(title = "income, ML estimate logN"))
p5

In [38]:
import bisect
binno = 50  # number of bins
lx_hist = np.histogram(fes_data_sel['lincome'], bins=binno, density=True)
lx_hist_bp = lx_hist[1][1:-1]  # remove first and last 
lx_hist_den = lx_hist[0]
temp2 = [bisect.bisect_left(lx_hist_bp, x) for x in lx_pdf_plot['lincome']] # identifies the right bin for each value in `lx`.
lx_pdf_plot['hist'] = lx_hist_den[temp2]    # now finds the right bin height

In [39]:
p6 = (ggplot(lx_pdf_plot) + 
        geom_line(aes(x = 'lincome', y = 'lx_pdf')) +
        geom_area(aes(x = 'lincome', y = 'hist'), color = "red", fill = "orange", alpha = 0.2) +
        labs(title = "Distribution, income", x = "log(income)", y = "Density"))
p6

In [41]:
binno = 100  # number of bins
x_hist = np.histogram(fes_data_sel['income'], bins=binno, density=True)
x_hist_bp = x_hist[1][1:-1]  # remove first and last 
x_hist_den = x_hist[0]
temp2 = [bisect.bisect_left(x_hist_bp, x) for x in x_pdf_plot['income']] # identifies the right bin for each value in `lx`.
x_pdf_plot['hist'] = x_hist_den[temp2]    # now finds the right bin height

In [43]:
p7 = (ggplot(x_pdf_plot) + 
        geom_line(aes(x = 'income', y = 'x_pdf'), color = "black") +
        geom_area(aes(x = 'income', y = 'hist'), color = "red", fill = "orange", alpha = 0.2) +
        labs(title = "Distribution, income", x = "income", y = "Density"))
p7

In [46]:
from sklearn.neighbors import KernelDensity

# instantiate and fit the KDE model
kde = KernelDensity(bandwidth=1.0, kernel='gaussian')
# feed in the observed data, requires 2D object which you can get from dataframe using dataframe[['VarName']]
kde.fit(fes_data_sel[['income']]) 

# score_samples returns the log of the probability density for all values of x
temp = kde.score_samples(x_pdf_plot[['income']])
# temp contains the log-likelihood and we need to calculate np.exp(temp) to to get the density
x_pdf_plot['x_pdf_kde_normal_bw1'] = np.exp(temp) 

In [48]:
p8 = (ggplot(x_pdf_plot) + 
        geom_line(aes(x = 'income', y = 'x_pdf'), color = "black") +
        geom_line(aes(x = 'income', y = 'x_pdf_kde_normal_bw1'), color = "cyan") +
        geom_area(aes(x = 'income', y = 'hist'), color = "red", fill = "orange", alpha = 0.2) +
        labs(title = "Distribution, income", x = "income", y = "Density"))
p8

In [49]:
kde = KernelDensity(bandwidth=100.0, kernel='gaussian')
# noe feed in the observed data, requires 2D object which you can get fram dataframe using dataframe[['VarName']]
kde.fit(fes_data_sel[['income']]) 

# score_samples returns the log of the probability density for all values of x
temp = kde.score_samples(x_pdf_plot[['income']])
# temp contains the log-likelihood and we need to calculate np.exp(temp) to to get the density
x_pdf_plot['x_pdf_kde_normal_bw100'] = np.exp(temp) 

In [51]:
p9 = (ggplot(x_pdf_plot) + 
        geom_line(aes(x = 'income', y = 'x_pdf'), color = "black") +
        geom_line(aes(x = 'income', y = 'x_pdf_kde_normal_bw1'), color = "cyan") +
        geom_line(aes(x = 'income', y = 'x_pdf_kde_normal_bw100'), color = "blue") +
        geom_area(aes(x = 'income', y = 'hist'), color = "red", fill = "orange", alpha = 0.2) +
        labs(title = "Distribution, income", x = "income", y = "Density"))
p9

In [52]:
sigmahat = np.std(fes_data_sel['income'])
n = len(fes_data_sel['income']) - sum(fes_data_sel['income'].isna()) # checking for missing observations
hopt = 1.06 * sigmahat * n **(-0.2)

print("Optimal bandwidth (Silverman), income = ", round(hopt,4))

Optimal bandwidth (Silverman), income =  63.4507


In [53]:
kde = KernelDensity(bandwidth=hopt, kernel='gaussian')
kde.fit(fes_data_sel[['income']]) 
temp = kde.score_samples(x_pdf_plot[['income']])
x_pdf_plot['x_pdf_kde_normal_bwopt'] = np.exp(temp) 

In [54]:
p10 = (ggplot(x_pdf_plot) + 
        geom_line(aes(x = 'income', y = 'x_pdf_kde_normal_bw1'), color = "cyan") +
        geom_line(aes(x = 'income', y = 'x_pdf_kde_normal_bw100'), color = "blue") +
        geom_line(aes(x = 'income', y = 'x_pdf_kde_normal_bwopt'), color = "red") +
        labs(title = "Distribution, income", x = "income", y = "Density"))
p10

In [55]:
sigmahat = np.std(fes_data_sel['lincome'])
n = len(fes_data_sel['lincome']) - sum(fes_data_sel['lincome'].isna()) # checking for missing observations
hopt = 1.06 * sigmahat * n **(-0.2)

print("Optimal bandwidth (Silverman), log(income) = ", round(hopt,4))

kde = KernelDensity(bandwidth=hopt, kernel='gaussian')
kde.fit(fes_data_sel[['lincome']]) 
temp = kde.score_samples(lx_pdf_plot[['lincome']])
lx_pdf_plot['lx_pdf_kde_normal_bwopt'] = np.exp(temp) 

p11 = (ggplot(lx_pdf_plot) + 
        geom_line(aes(x = 'lincome', y = 'lx_pdf'), color = "red") +
        geom_line(aes(x = 'lincome', y = 'lx_pdf_kde_normal_bwopt'), color = "blue") +
        geom_area(aes(x = 'lincome', y = 'hist'), color = "orange", fill = "orange", alpha = 0.2) +

        labs(title = "Distribution, income", x = "ln(income)", y = "Density"))
p11

Optimal bandwidth (Silverman), log(income) =  0.0876


In [56]:
sigmahat = np.std(fes_data_sel['lincome'])
iqr = np.percentile(fes_data_sel['lincome'],75)-np.percentile(fes_data_sel['lincome'],25)

n = len(fes_data_sel['lincome']) - sum(fes_data_sel['lincome'].isna()) # checking for missing observations
hopt = 1.06 * np.min([sigmahat,iqr/1.34]) * n **(-0.2)

print("Plug-in bandwidth (Ebanechnikov), log(income) = ", round(hopt,4))

kde = KernelDensity(bandwidth=hopt, kernel='epanechnikov')
kde.fit(fes_data_sel[['lincome']]) 
temp = kde.score_samples(lx_pdf_plot[['lincome']])
lx_pdf_plot['lx_pdf_kde_epa_bwopt'] = np.exp(temp) 

p11 = (ggplot(lx_pdf_plot) + 
        geom_line(aes(x = 'lincome', y = 'lx_pdf_kde_epa_bwopt'), color = "red") +
        geom_line(aes(x = 'lincome', y = 'lx_pdf_kde_normal_bwopt'), color = "blue") +
        geom_area(aes(x = 'lincome', y = 'hist'), color = "orange", fill = "orange", alpha = 0.2) +
        xlim(3,9) +
        labs(title = "Distribution, income", x = "ln(income)", y = "Density"))
p11

Plug-in bandwidth (Ebanechnikov), log(income) =  0.0732


In [60]:
bw_sel = np.linspace(0.05, 0.40, 20)
bw_sel

array([0.05      , 0.06842105, 0.08684211, 0.10526316, 0.12368421,
       0.14210526, 0.16052632, 0.17894737, 0.19736842, 0.21578947,
       0.23421053, 0.25263158, 0.27105263, 0.28947368, 0.30789474,
       0.32631579, 0.34473684, 0.36315789, 0.38157895, 0.4       ])

In [61]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

# This procedure can take a lot of time
# for testing we take a random sample of 1000, fes_data_sel[['lincome']].sample(1000)
# if you wish to run on all data delete `.sample(1000)`
# WARNING: will take a few minutes

kde = KernelDensity(kernel='gaussian')
grid = GridSearchCV(kde, {'bandwidth': bw_sel}, cv = 3)
grid.fit(fes_data_sel[['lincome']].sample(1000))

GridSearchCV(cv=3, estimator=KernelDensity(),
             param_grid={'bandwidth': array([0.05      , 0.06842105, 0.08684211, 0.10526316, 0.12368421,
       0.14210526, 0.16052632, 0.17894737, 0.19736842, 0.21578947,
       0.23421053, 0.25263158, 0.27105263, 0.28947368, 0.30789474,
       0.32631579, 0.34473684, 0.36315789, 0.38157895, 0.4       ])})

In [62]:
kde_best = grid.best_estimator_
# Print the best bandwidth
print("Best bandwidth: ", round(kde_best.bandwidth,4))

Best bandwidth:  0.2342


In [63]:
temp = kde_best.score_samples(lx_pdf_plot[['lincome']])
lx_pdf_plot['lx_pdf_kde_normal_bwcv'] = np.exp(temp) 

In [64]:
p12 = (ggplot(lx_pdf_plot) + 
        geom_line(aes(x = 'lincome', y = 'lx_pdf_kde_normal_bwopt'), color = "red") +
        geom_line(aes(x = 'lincome', y = 'lx_pdf_kde_normal_bwcv'), color = "blue") +
        xlim(3,9) +
        labs(title = "Distribution, income", x = "ln(income)", y = "Density"))
p12